# Manipulación y preparación de datos
# Por Francisco Antonio Enríquez Cabrera (A01795006) (Ing. de datos)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings

In [2]:
# Ignorar advertencias innecesarias
warnings.filterwarnings('ignore')

In [3]:
# Cargar el dataset
file_path = f'../data/raw/{"forest_fires_df.csv"}'
data = pd.read_csv(file_path)

In [4]:
# Visualizar las primeras filas del dataset
print("Primeras filas del dataset:")
print(data.head())

Primeras filas del dataset:
   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0


In [5]:
# Información del dataset para verificar los tipos de datos y valores nulos
print("\nInformación del dataset:")
data.info()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [6]:
# Revisar si hay valores nulos
print("\nValores nulos por columna:")
print(data.isnull().sum())


Valores nulos por columna:
X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
area     0
dtype: int64


In [7]:
# Corregir potenciales valores nulos si los hay
data = data.dropna()

In [8]:
# Revisar y Eliminar duplicados
print(f"\nNúmero de filas duplicadas: {data.duplicated().sum()}")
data = data.drop_duplicates()
print(f"Número de filas después de eliminar duplicados: {data.shape[0]}")


Número de filas duplicadas: 4
Número de filas después de eliminar duplicados: 513


In [9]:
# Detectar y manejar outliers en la variable 'area' usando el método del rango intercuartílico (IQR)
Q1 = data['area'].quantile(0.25)
Q3 = data['area'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [10]:
# Filtrar las filas con valores fuera de los límites
data_filtered = data[(data['area'] >= lower_bound) & (data['area'] <= upper_bound)]
print(f"Cantidad de datos después de eliminar outliers: {data_filtered.shape[0]}")

Cantidad de datos después de eliminar outliers: 451


In [11]:
# Transformar la variable 'area' (debido al sesgo hacia valores pequeños) usando una escala logarítmica log(1 + area) para mejorar la distribución
data_filtered['log_area'] = np.log1p(data_filtered['area'])

In [12]:
# Codificar variables categóricas usando One-Hot Encoding para 'month' y 'day'
categorical_features = ['month', 'day']
numerical_features = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']

In [13]:
# Definir el pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Escalar características numéricas
        ('cat', OneHotEncoder(drop='first'), categorical_features)  # One-Hot Encoding para categóricas
    ])

In [53]:
# Separar las características y la variable objetivo
X = data_filtered.drop(columns=['area', 'log_area'])  # Características predictoras (sin 'area' ni 'log_area')
y = data_filtered['log_area']  # Variable objetivo (área quemada en escala logarítmica)

In [54]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
# Crear el pipeline de transformación y modelado con Random Forest
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [56]:
# Entrenar el modelo de Machine Learning (Random Forest Regressor)
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['X', 'Y', 'FFMC', 'DMC',
                                                   'DC', 'ISI', 'temp', 'RH',
                                                   'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [57]:
# Realizar predicciones sobre el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

In [58]:
# Evaluar el modelo con métricas de rendimiento
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Root Mean Squared Error (RMSE)
r2 = r2_score(y_test, y_pred)  # Coeficiente de determinación R²

In [59]:
# Mostrar los resultados de las métricas de evaluación del modelo
print("\nEvaluación del modelo:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")


Evaluación del modelo:
RMSE: 0.9946
R²: -0.1767


In [60]:
# Guardar los datos procesados en archivos CSV para su uso posterior
X_train.to_csv('X_train_processed.csv', index=False)
X_test.to_csv('X_test_processed.csv', index=False)
y_train.to_csv('y_train_processed.csv', index=False)
y_test.to_csv('y_test_processed.csv', index=False)

print("\nLos datos han sido procesados y guardados en archivos CSV.")


Los datos han sido procesados y guardados en archivos CSV.


In [14]:
# Guardar el dataset final preparado
file_path = f'../data/processed/{"forest_fires_prepared_df.csv"}'
data_filtered.to_csv(file_path, index=False)